In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import re

In [3]:
las_vegas = pd.read_csv('data/Las_Vegas_user_groupby_review.csv')
#las_vegas_all_reviews = pd.read_csv('data/Las_Vegas_review.csv')
#reviews = pd.read_csv('data/reviews.csv')
business = pd.read_csv('data/business.csv')

In [ ]:
cities = ['Las Vegas','Phoenix','Toronto']
business.dropna(inplace = True)
restaurants_business = business[business.categories.str.contains("Restaurants")]
business_city = restaurants_business[restaurants_business.city == 'Las Vegas']
bus_rev_city = business_city.merge(reviews, on = 'business_id')
bus_rev_city = bus_rev_city.drop(columns = ['Unnamed: 0'])

### Preprocess for TF-IDF

In [6]:
las_vegas_all_reviews

,name,business_id,city,categories,review_id,text,stars,user_id
0,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",WUAt5-krh075Ie44np8Hew,So why would I be giving a Fast Food chain loc...,5.0,YDkC5VVT8s9NlIZtAr8NUA
1,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",S9vIs-je49jqbhpWSQ8yHg,I come here bout 3x's a mo. & I just can't do ...,2.0,mxtalrN7VHVQwGjv8ln5yg
2,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",i2Soid_X8XsSTyIrPOst6g,WORST experience EVER!!!!! never have i ate an...,1.0,xammeA3ftpFoAN13v5Blyg
3,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",uDYGaAOUgo7SLuH5nUZs3w,Hot fresh food usually. Staff seems to turn o...,4.0,gQhFacMGI41mfVXqw1E7bQ
4,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",Xg7HuBvuZHTBog1G_2Wk0w,Beer battered cod fish seems really rubbery. I...,3.0,0PXhJMztE3ijzQPjiwPERg
...,...,...,...,...,...,...,...,...
1503839,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",3TD_bRi98t5UDLuesY66Hw,Came to Vegas with the fam and some other fami...,1.0,UjBiVw0WFmEbgK6qgap3QA
1503840,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",IeowybrKAmmf9njf7pa1rA,So there are alot of pretty bad reviews but I ...,3.0,F5YcdVMqhDAfhGZsMCXDTA
1503841,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",r7maiBTmlfFRcaLAmwY0Aw,All the buffets are pretty much the same. Ove...,2.0,CPLndzPfGfJzjqjzXCr9_g
1503842,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",UPn6fAXxo7SEaQL4xkUgFA,Don't come here for dinner. You can see the b...,1.0,pSoe4XBftnJiTgG1fPw52g


In [18]:
def preprocess_group_user_review(review_df):
    user_df = review_df[['text','categories','user_id']].groupby('user_id').agg({'text':list,
                                                                            'categories':lambda column: ' '.join(column)}
                                                                          ).reset_index()
    user_df.text = user_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return user_df

In [27]:
def preprocess_group_restaurant_review(review_df, business_df):
    restaurant_df = review_df[['text','stars','business_id']].groupby('business_id').agg({'text':list,
                                                                                          'stars':np.mean}
                                                                                        ).reset_index()
    restaurant_df.text = restaurant_df.text.apply(lambda t: "".join(re.sub(r'[^\w\s]',' ',str(t))).replace("\n"," "))
    return restaurant_df.merge(business_df[['name', 'business_id']], on = 'business_id')

In [44]:
joined_user = preprocess_group_user_review(a)
joined_user

,user_id,text,categories
0,--RlSfc-QmcHFGHyX6aVjA,HATE Nothing n nDISLIKE Door had too many...,"Restaurants, Specialty Food, Japanese, Sushi B..."
1,--i9JYrfaKvCpI60nSyP3Q,So I totally had to check this place out I ...,"Sandwiches, Shaved Ice, Coffee & Tea, Desserts..."
2,-0iBCjlEgpPZ5QVE8QzOrA,I have been a customer since the 1980 s an...,"Sandwiches, Italian, Restaurants, Seafood"
3,-16aKjco1c0RJ7c4U-q_Kw,Wow Veggie House is on the top of my list o...,"Restaurants, Specialty Food, Japanese, Sushi B..."
4,-1KoTYfeKl8ZH225pBe4Ew,AMAZING I would definitely recommend this...,"Sandwiches, Italian, Restaurants, Seafood"
...,...,...,...
9020,zwecaiPFy12UI2ppTePy3w,Great place Awesome sushi sashimi Servi...,"Teppanyaki, Japanese, Restaurants, Steakhouses..."
9021,zx26BmC67gLmQrAgY-nT1g,Went last night for early dinner before a sh...,"Sandwiches, Italian, Restaurants, Seafood"
9022,zx5rdBK9NFZrAglIXF1LiQ,Locos taco is amazing,"American (New), Restaurants, Food, Tex-Mex, Ta..."
9023,zyMB50UtmEuQf-1VUKxZxQ,This place is legit I ve been here 3 times ...,"Vietnamese, Restaurants, American (Traditional)"


In [46]:
joined_restaurant = preprocess_group_restaurant_review(a, business)
joined_restaurant

,business_id,text,stars,name
0,-wDNRL4lm1UgeFLG9KcrjQ,Pretty good fast Asian food Nice service A...,3.730233,Yummy Grill & Sushi
1,4Nj2ktP2NjCbv2aKeH3ZBg,This was by far our best coffee experience i...,4.560000,Vesta Coffee Roasters
2,6e0Abng1LRexKI4LRJTX_A,Just tried this place for the first time and...,4.120301,Algoberto’s
3,7vWb1Afpm2arr8NCaE_EyQ,These guys are exceptional We shot a video...,4.444444,Ogden Mart
4,AN0bWhisCf6LN9eHZ7DQ3w,Very good restaurant they have many choices...,4.834025,Los Olivos Ristorante
...,...,...,...,...
68,wlcHT4d52xh52ZsIxxuGFg,This was my first visit EVER to Smashburger ...,3.310680,Smashburger
69,wnxxb-Qpy5sYWCKIKto5Tg,The Rampart Hotel actually found a way to ta...,3.363636,Market Place Buffet
70,x1fx7C_tc064cFATjl4fmQ,Came here for my billiards league and found ...,3.120968,The Lodge at Grand Teton
71,xPuId11JJs0uy-QuPfifOw,First time here Quiet Thursday night Mosco...,3.137931,Grill 55


### Convert the text review dataframe into the TF-IDF featue matrix

In [47]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(test_joined_restaurant.text.apply(lambda x: x.lower()))

### User - User Simple TF-IDF and Cosine Similarity recommendation

In [60]:
def user_recommendations(ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        User ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).head(k)

In [22]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)

In [23]:
user_user_df = pd.DataFrame(cosine_sim, columns=t.user_id, index=t.user_id)

In [62]:
# given a user ID and the similarity matrix, recommend a list of similar user
user_id = '---1lKK3aKOuomHnwAkAow' # input user id
add_on = t[['user_id','text', 'categories']]

user_recommendations(user_id, user_user_df, add_on, k=5)

,user_id,text,categories
0,-50XWnmQGqBgEI-9ANvLlg,Experience Loved nA group of friends met t...,"Restaurants, Burgers, Nightlife, American (Tra..."
1,-267Yx8RmdP6io2-qI4UcQ,The chef here is nothing short of a magician...,"Restaurants, Specialty Food, Japanese, Sushi B..."
2,-4JDJeFS0YAYSiSvIshGLQ,No problem with the quality or preparation o...,"Restaurants, Sushi Bars, Buffets, Japanese, Ba..."
3,-594af_E7Z9VVjQc9pJK3g,Things I love about Sidesteeet Pho and Grill...,"Vietnamese, Restaurants, American (Traditional..."
4,-21ooEUDqBkCF0hr94Pvgg,So here s the low down with this new hot spo...,"Restaurants, Japanese, Cafes Restaurants, Vege..."


### Resraurant - Reataurant TF-IDF approach

In [57]:
def restaurant_recommendations(business_ID, df, record, k=10):
    """
    Recommends user based on a similarity dataframe
    Parameters
    ----------
    ID : str
        Restaurant Business ID (index of the similarity dataframe)
    df : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    record : pd.DataFrame
        The record of original user and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    ix = df.loc[:,business_ID].to_numpy().argpartition(range(-1,-k,-1))
    high_recomm = df.columns[ix[-1:-(k+2):-1]]
    high_recomm = high_recomm.drop(business_ID, errors='ignore')
    return pd.DataFrame(high_recomm).merge(record).drop_duplicates().head(k)

In [50]:
# direct calculate the cosine similarity and generate the dataframe
cosine_sim = cosine_similarity(tfidf_matrix)
rest_rest_df = pd.DataFrame(cosine_sim, columns=joined_restaurant.business_id, index=joined_restaurant.business_id)

In [58]:
# given a user ID and the similarity matrix, recommend a list of similar user
business_id = '4Nj2ktP2NjCbv2aKeH3ZBg' # input business id
add_on = a[['business_id','categories']]

restaurant_recommendations(business_id, rest_rest_df, add_on, k=5)

,business_id,categories
0,RIHydChuDTrfuuvr8TzH4g,"Restaurants, Cafes"
6,oUX2bYbqjqST-urKbOHG6w,"Sandwiches, Shaved Ice, Coffee & Tea, Desserts..."
297,c1JoHp602zilpDU_57DsMg,"Restaurants, Cafes, Food, Internet Cafes, Coff..."
356,MNaPOkqm2vKeNG8EqkZWCg,"Sandwiches, Italian, Restaurants, Seafood"
849,oaqbLDW3wNqkynlnp4mvlw,"Italian, Restaurants"


### Other Try

In [ ]:
for i in tqdm(range(len(cities))):
    business_city = restaurants_business[restaurants_business.city == cities[i]]
    bus_rev_city = business_city.merge(reviews, on = 'business_id')

In [ ]:
def calc_sim_user_rating(target_user_id, target_business_id, groupby_df, original_df):
    business_id_df = original_df[original_df.business_id == target_business_id].copy()
    
    target_user_tfidf = tfidf_matrix[groupby_df[groupby_df.user_id == target_user_id].index[0]].copy()
    
    dist_user = []
    for user_id in business_id_df.user_id.values:
        dist_user.append(cosine_similarity(tfidf_matrix[groupby_df[groupby_df.user_id==user_id].index[0]],
                                           target_user_vector)[0][0])
        
    business_id_df['similarity'] = dist_user    
    #Filtering technique = Calc percentile and filter percentiles > 80%
    business_id_df = business_id_df[business_id_df.rank(pct=True).similarity > .8]
     
    rating_diff = []
    for user_id in business_id_df.user_id.values:
        rating_diff.append(test_rest_user_id.stars_review[test_rest_user_id.user_id==user_id].values[0]
                            -np.average(city_train[city_train.user_id ==user_id].stars_review.values))
    test_rest_user_id['rating_diff'] = rating_diff
    
    return np.average(test_rest_user_id.rating_diff)

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in test.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], test['user_id'][i]) for i in similar_indices]

    results[row['user_id']] = similar_items[1:]
    
print('done!')

def item(id):
    return test.loc[test['user_id'] == id]['text'].tolist()[0].split(' - ')[0]

# Just reads the results out of the dictionary.
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")